In [1]:
using LinearAlgebra, Distributions

In [2]:
one_vec = ones(2);
zero_vec = zeros(2);

### MAP creation

In [3]:
function check_is_MAP(C,D)
    all(D .>= 0.0) || return false
    all(diag(C) .< 0.0) || return false
    all(C-Diagonal(C) .>= 0.0) ||return false
    iszero(sum(C+D,dims=2)) || return false   #will be redundent in the make_C_D function and in the similarity...
    return true
end;

$$
\boldsymbol{\alpha} = 
\frac{1}{(\lambda_{21}+ \lambda_{22}) q_{12}+ (\lambda_{11}+ \lambda_{12}) q_{21}}
\Big[q_{21} \lambda_{11} + q_{12} \lambda_{21}, ~~
q_{21} \lambda_{12} + q_{12} \lambda_{22}
 \Big].
 $$


In [4]:
function make_C_D_α(q12, q21, λ11, λ12, λ21, λ22)
    C = [
        -q12 - λ11      q12 - λ12;
         q21 - λ21   -q21 - λ22
    ]
    D = [
        λ11  λ12;
        λ21  λ22
    ]
    check_is_MAP(C,D) || error("problem with parameters")

    α = ([q21*λ11 + q12*λ21, q21*λ12 + q12*λ22]') / ((λ21 + λ22)*q12 + (λ11 + λ12)*q21)
    return C, D, α
end

q12 = 10
q21 = 20
λ11 = 1.0
λ12 = 2.0
λ21 = 3.0
λ22 = 4.0

C, D, α = make_C_D_α(q12, q21, λ11, λ12, λ21, λ22);

In [5]:
C

2×2 Matrix{Float64}:
 -11.0    8.0
  17.0  -24.0

In [6]:
D

2×2 Matrix{Float64}:
 1.0  2.0
 3.0  4.0

In [7]:
Q = C+D

2×2 Matrix{Float64}:
 -10.0   10.0
  20.0  -20.0

In [11]:
Q*one_vec

2-element Vector{Float64}:
 0.0
 0.0

In [12]:
P = inv(-C)*D

2×2 Matrix{Float64}:
 0.375     0.625
 0.390625  0.609375

In [13]:
sum(P, dims = 2)

2×1 Matrix{Float64}:
 0.9999999999999998
 1.0

In [15]:
α*P - α #indeed stationary vector

1×2 adjoint(::Vector{Float64}) with eltype Float64:
 -5.55112e-17  0.0

### Similarity matrix

$$
S {\mathbf 1} = {\mathbf 1}
$$
so also,

$$
{\mathbf 1} = S^{-1}{\mathbf 1}
$$


In [16]:
function make_rand_S_matrix()
    s1, s2 = rand(Uniform(-1,1), 2)
    S = [s1 1-s1
         1-s2 s2]
    return S, inv(S)
end;

In [17]:
S, Sinv = make_rand_S_matrix();

In [18]:
S

2×2 Matrix{Float64}:
 -0.888162   1.88816
  1.44107   -0.441067

In [19]:
Sinv

2×2 Matrix{Float64}:
 0.189362  0.810638
 0.618688  0.381312

In [20]:
S*one_vec

2-element Vector{Float64}:
 1.0
 1.0

In [21]:
Sinv*one_vec

2-element Vector{Float64}:
 1.0
 1.0

### Transform MAP

$$
C_{\text{new}} = S^{-1} C S
$$


$$
D_{\text{new}} = S^{-1} D S
$$

$$
Q_{\text{new}} = C_{\text{new}} + D_{\text{new}} = S^{-1} (C + D) S
$$

So $Q_{\text{new}} {\mathbf 1} = 0$.

$$
\pi Q = 0
$$

Claim:

$$
\pi S Q_{\text{new}} = 0.
$$

Hence $\pi S$ is the $\pi_{\text{new}}$.

$$
\lambda^* = \pi D {\mathbf 1}
$$

$$
\lambda^*_{\text{new}} = \pi_{\text{new}} D_{\text{new}} {\mathbf 1} = \pi S S^{-1} D S {\mathbf 1} = \lambda^*
$$


$$
\alpha = \frac{1}{\lambda^*} \pi D
$$

$$
\alpha_{\text{new}} = \frac{1}{\lambda^*_{\text{new}}} \pi_{\text{new}} D_{\text{new}}
= \frac{1}{\lambda^*} \pi S S^{-1} D S = \alpha S
$$

In [22]:
function find_similar_map(C,D; max_attempts = 10^3)
    attempts = 0
    while attempts <= max_attempts
        S, Sinv = make_rand_S_matrix()
        Cnew = Sinv*C*S
        Dnew = Sinv*D*S
        is_map = check_is_MAP(Cnew, Dnew)
        if is_map
            @info "Found similar map after $attempts attempts"
            return S, Sinv, Cnew, Dnew
        end
        attempts += 1
    end
    error("did not find a similar one")
end;

In [23]:
S, Sinv, Cnew, Dnew = find_similar_map(C,D);

[ Info: Found similar map after 15 attempts


In [24]:
check_is_MAP(Cnew, Dnew)

true

In [25]:
Qnew = Cnew+Dnew

2×2 Matrix{Float64}:
 -19.7933   19.7933
  10.2067  -10.2067

In [28]:
sum(Qnew, dims = 2)

2×1 Matrix{Float64}:
 0.0
 0.0

In [27]:
Pnew = inv(-Cnew)*Dnew

2×2 Matrix{Float64}:
 0.752396  0.247604
 0.768021  0.231979

In [29]:
sum(Pnew, dims = 2)

2×1 Matrix{Float64}:
 1.0
 1.0

In [31]:
αnew = α*S

1×2 adjoint(::Vector{Float64}) with eltype Float64:
 0.756205  0.243795

In [32]:
αnew*Pnew-αnew

1×2 adjoint(::Vector{Float64}) with eltype Float64:
 1.11022e-16  8.32667e-17

### Actual probablity law (check on first two intervals)

In [36]:
pdf_original_1(t) = α*exp(C*t)*D*one_vec
pdf_new_1(t) = αnew*exp(Cnew*t)*Dnew*one_vec;

In [42]:
maximum(abs, [pdf_original_1(t) - pdf_new_1(t) for t in 0:0.01:2.5])

2.886579864025407e-15

In [41]:
pdf_original_2(t0, t1) = α*exp(C*t0)*D*exp(C*t1)*D*one_vec
pdf_new_2(t0, t1) = αnew*exp(Cnew*t0)*Dnew*exp(Cnew*t1)*Dnew*one_vec;

In [46]:
maximum(abs, [pdf_original_2(t0, t1) - pdf_new_2(t0, t1) for t0 in 0:0.01:2.5 for t1 in 0:0.01:2.5])

1.5987211554602254e-14